In the first cell, run the following command to install the Google Cloud Vertex AI, Cloud Storage and BigQuery SDKs. To run the command, execute SHIFT+ENTER

! pip3 install --upgrade google-cloud-aiplatform \
                        google-cloud-storage \
                        'google-cloud-bigquery[pandas]'

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)


{'status': 'ok', 'restart': True}

In [1]:
PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-west1"

In [2]:
import vertexai
vertexai.init(project = PROJECT_ID,
              location = REGION)

In [3]:
import math
from typing import Any, Generator

import pandas as pd
from google.cloud import bigquery

client = bigquery.Client(project=PROJECT_ID)

In [4]:
QUERY_TEMPLATE = """
        SELECT distinct q.id, q.title, q.body
        FROM (SELECT * FROM `bigquery-public-data.stackoverflow.posts_questions` where Score>0 ORDER BY View_Count desc) AS q
        LIMIT {limit} OFFSET {offset};
        """

In [5]:
def query_bigquery_chunks(
    max_rows: int, rows_per_chunk: int, start_chunk: int = 0
) -> Generator[pd.DataFrame, Any, None]:
    for offset in range(start_chunk, max_rows, rows_per_chunk):
        query = QUERY_TEMPLATE.format(limit=rows_per_chunk, offset=offset)
        query_job = client.query(query)
        rows = query_job.result()
        df = rows.to_dataframe()
        df["title_with_body"] = df.title + "\n" + df.body
        yield df

In [6]:
df = next(query_bigquery_chunks(max_rows=1000, rows_per_chunk=1000))

# Examine the data
df.head()

,id,title,body,title_with_body
0,25836860,UI flicker while pressing back in android,<p>I am creating a UI for my company which has...,UI flicker while pressing back in android\n<p>...
1,8193953,TIdHTTPServer connection aborted,<p>I have a http server application and I'm us...,TIdHTTPServer connection aborted\n<p>I have a ...
2,35754588,How to use abstract classes inside a UserControl,<p>I have a <code>UserControl</code>.</p>\n\n<...,How to use abstract classes inside a UserContr...
3,35708677,UILocalNotification triggers daily even after ...,<p>I am trying to create a local notification ...,UILocalNotification triggers daily even after ...
4,2931053,Tkinter Label bound to StringVar is one click ...,"<p>The problem I'm running into here is that, ...",Tkinter Label bound to StringVar is one click ...


In [7]:
from typing import List, Optional
from vertexai.preview.language_models import TextEmbeddingModel

model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

In [8]:
def encode_texts_to_embeddings(sentences: List[str]) -> List[Optional[List[float]]]:
    try:
        embeddings = model.get_embeddings(sentences)
        return [embedding.values for embedding in embeddings]
    except Exception:
        return [None for _ in range(len(sentences))]

In [9]:
import functools
import time
from concurrent.futures import ThreadPoolExecutor
from typing import Generator, List, Tuple

import numpy as np
from tqdm.auto import tqdm


# Generator function to yield batches of sentences
def generate_batches(
    sentences: List[str], batch_size: int
) -> Generator[List[str], None, None]:
    for i in range(0, len(sentences), batch_size):
        yield sentences[i : i + batch_size]

In [10]:
def encode_text_to_embedding_batched(
    sentences: List[str], api_calls_per_second: int = 10, batch_size: int = 5
) -> Tuple[List[bool], np.ndarray]:

    embeddings_list: List[List[float]] = []

    # Prepare the batches using a generator
    batches = generate_batches(sentences, batch_size)

    seconds_per_job = 1 / api_calls_per_second

    with ThreadPoolExecutor() as executor:
        futures = []
        for batch in tqdm(
            batches, total=math.ceil(len(sentences) / batch_size), position=0
        ):
            futures.append(
                executor.submit(functools.partial(encode_texts_to_embeddings), batch)
            )
            time.sleep(seconds_per_job)

        for future in futures:
            embeddings_list.extend(future.result())

    is_successful = [
        embedding is not None for sentence, embedding in zip(sentences, embeddings_list)
    ]
    embeddings_list_successful = np.squeeze(
        np.stack([embedding for embedding in embeddings_list if embedding is not None])
    )
    return is_successful, embeddings_list_successful

In [11]:
# Encode a subset of questions for validation
questions = df.title.tolist()[:500]
is_successful, question_embeddings = encode_text_to_embedding_batched(
    sentences=df.title.tolist()[:500]
)

# Filter for successfully embedded sentences
questions = np.array(questions)[is_successful]

  0%|          | 0/100 [00:00<?, ?it/s]

In [12]:
DIMENSIONS = len(question_embeddings[0])

print(DIMENSIONS)

768


In [13]:
import random

question_index = random.randint(0, 99)

print(f"Query question = {questions[question_index]}")

# Get similarity scores for each embedding by using dot-product.
scores = np.dot(question_embeddings[question_index], question_embeddings.T)

# Print top 20 matches
for index, (question, score) in enumerate(
    sorted(zip(questions, scores), key=lambda x: x[1], reverse=True)[:20]
):
    print(f"\t{index}: {question}: {score}")

Query question = Selecting rows in a DBSet with Entity Framework
	0: Selecting rows in a DBSet with Entity Framework: 0.999999366169285
	1: Entity Framework with Linq, inner Join, Group By, Order By: 0.716863218168867
	2: Linq distinct record containing keywords: 0.6647837532488451
	3: How can I pass a linq var datatype to a method?: 0.6418270731916208
	4: How to convert IEnumerable<IEnumerable<T>> to List<string>?: 0.6309984170983918
	5: How to sort list on multiple properties in one line of code in Linq: 0.6305464122669026
	6: mvc4 - db.SaveChanges() - decimal value out of range: 0.6180778212242855
	7: Using MVC2 to update an Entity Framework v4 object with foreign keys fails: 0.6105349705170884
	8: DropDownList Binding to ActionResult Create Method MVC 4 VS2012: 0.5935508733819446
	9: My C# Settings method, not working to well: 0.5911589782643845
	10: Offset Fetch Next to get all rows: 0.5832571980311714
	11: Sorting OQL result set by object parameter in VisualVm: 0.5817505606136242

In [14]:
import tempfile
from pathlib import Path

# Create temporary file to write embeddings to
embeddings_file_path = Path(tempfile.mkdtemp())

print(f"Embeddings directory: {embeddings_file_path}")

Embeddings directory: /tmp/tmpuq_9rqm7


In [15]:
import gc
import json

BQ_NUM_ROWS = 5000
BQ_CHUNK_SIZE = 1000
BQ_NUM_CHUNKS = math.ceil(BQ_NUM_ROWS / BQ_CHUNK_SIZE)

START_CHUNK = 0

# Create a rate limit of 300 requests per minute. Adjust this depending on your quota.
API_CALLS_PER_SECOND = 300 / 60
# According to the docs, each request can process 5 instances per request
ITEMS_PER_REQUEST = 5

# Loop through each generated dataframe, convert
for i, df in tqdm(
    enumerate(
        query_bigquery_chunks(
            max_rows=BQ_NUM_ROWS, rows_per_chunk=BQ_CHUNK_SIZE, start_chunk=START_CHUNK
        )
    ),
    total=BQ_NUM_CHUNKS - START_CHUNK,
    position=-1,
    desc="Chunk of rows from BigQuery",
):
    # Create a unique output file for each chunk
    chunk_path = embeddings_file_path.joinpath(
        f"{embeddings_file_path.stem}_{i+START_CHUNK}.json"
    )
    with open(chunk_path, "a") as f:
        id_chunk = df.id

        # Convert batch to embeddings
        is_successful, question_chunk_embeddings = encode_text_to_embedding_batched(
            sentences=df.title_with_body.to_list(),
            api_calls_per_second=API_CALLS_PER_SECOND,
            batch_size=ITEMS_PER_REQUEST,
        )

        # Append to file
        embeddings_formatted = [
            json.dumps(
                {
                    "id": str(id),
                    "embedding": [str(value) for value in embedding],
                }
            )
            + "\n"
            for id, embedding in zip(id_chunk[is_successful], question_chunk_embeddings)
        ]
        f.writelines(embeddings_formatted)

        # Delete the DataFrame and any other large data structures
        del df
        gc.collect()

Chunk of rows from BigQuery:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [16]:
BUCKET_URI = f"gs://{PROJECT_ID}-unique"

In [17]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://qwiklabs-gcp-03-de903c8f57bd-unique/...


In [18]:
remote_folder = f"{BUCKET_URI}/{embeddings_file_path.stem}/"
! gsutil -m cp -r {embeddings_file_path}/* {remote_folder}

Copying file:///tmp/tmpuq_9rqm7/tmpuq_9rqm7_0.json [Content-Type=application/json]...
Copying file:///tmp/tmpuq_9rqm7/tmpuq_9rqm7_1.json [Content-Type=application/json]...
Copying file:///tmp/tmpuq_9rqm7/tmpuq_9rqm7_2.json [Content-Type=application/json]...
Copying file:///tmp/tmpuq_9rqm7/tmpuq_9rqm7_4.json [Content-Type=application/json]...
Copying file:///tmp/tmpuq_9rqm7/tmpuq_9rqm7_3.json [Content-Type=application/json]...
\ [5/5 files][ 45.6 MiB/ 45.6 MiB] 100% Done                                    
Operation completed over 5 objects/45.6 MiB.                                     


In [19]:
DISPLAY_NAME = "stack_overflow"
DESCRIPTION = "question titles and bodies from stackoverflow"

In [21]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

DIMENSIONS = 768

tree_ah_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=DISPLAY_NAME,
    contents_delta_uri=remote_folder,
    dimensions=DIMENSIONS,
    approximate_neighbors_count=150,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    leaf_node_embedding_count=500,
    leaf_nodes_to_search_percent=80,
    description=DESCRIPTION,
)

Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/370509227950/locations/us-west1/indexes/6867831102065606656/operations/7528552927963643904
MatchingEngineIndex created. Resource name: projects/370509227950/locations/us-west1/indexes/6867831102065606656
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/370509227950/locations/us-west1/indexes/6867831102065606656')


In [22]:
INDEX_RESOURCE_NAME = tree_ah_index.resource_name
INDEX_RESOURCE_NAME

'projects/370509227950/locations/us-west1/indexes/6867831102065606656'

In [23]:
tree_ah_index = aiplatform.MatchingEngineIndex(index_name=INDEX_RESOURCE_NAME)

In [24]:
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=DISPLAY_NAME,
    description=DISPLAY_NAME,
    public_endpoint_enabled=True,
)

Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/370509227950/locations/us-west1/indexEndpoints/8650649624085790720/operations/5276753114278395904
MatchingEngineIndexEndpoint created. Resource name: projects/370509227950/locations/us-west1/indexEndpoints/8650649624085790720
To use this MatchingEngineIndexEndpoint in another session:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/370509227950/locations/us-west1/indexEndpoints/8650649624085790720')


In [25]:
DEPLOYED_INDEX_ID = "deployed_index_id_unique"

DEPLOYED_INDEX_ID


my_index_endpoint = my_index_endpoint.deploy_index(
    index=tree_ah_index, deployed_index_id=DEPLOYED_INDEX_ID
)

my_index_endpoint.deployed_indexes

Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/370509227950/locations/us-west1/indexEndpoints/8650649624085790720
Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/370509227950/locations/us-west1/indexEndpoints/8650649624085790720/operations/5326292710179471360
MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/370509227950/locations/us-west1/indexEndpoints/8650649624085790720


[]

In [26]:
number_of_vectors = sum(
    aiplatform.MatchingEngineIndex(
        deployed_index.index
    )._gca_resource.index_stats.vectors_count
    for deployed_index in my_index_endpoint.deployed_indexes
)

print(f"Expected: {BQ_NUM_ROWS}, Actual: {number_of_vectors}")

Expected: 5000, Actual: 0


In [27]:
test_embeddings = encode_texts_to_embeddings(sentences=["Install GPU for Tensorflow"])

In [28]:
NUM_NEIGHBOURS = 10

response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=test_embeddings,
    num_neighbors=NUM_NEIGHBOURS,
)

response

[[MatchNeighbor(id='54612450', distance=0.7736330032348633, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[]),
  MatchNeighbor(id='56660766', distance=0.7319902777671814, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[]),
  MatchNeighbor(id='54385264', distance=0.6886470317840576, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[]),
  MatchNeighbor(id='53423775', distance=0.6821916699409485, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[]),
  MatchNeighbor(id='65041694', distance=0.6779682636260986, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[]),
  MatchNeighbor(id='61817763', distance=0.6717808246612549, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[]),
  MatchNeighbor(id='53031277', distance=0.660767138004303, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[]),
  MatchNeighbor(id='63380464', distance=0.6577311754226685, fea

In [29]:
for match_index, neighbor in enumerate(response[0]):
    print(f"https://stackoverflow.com/questions/{neighbor.id}")

https://stackoverflow.com/questions/54612450
https://stackoverflow.com/questions/56660766
https://stackoverflow.com/questions/54385264
https://stackoverflow.com/questions/53423775
https://stackoverflow.com/questions/65041694
https://stackoverflow.com/questions/61817763
https://stackoverflow.com/questions/53031277
https://stackoverflow.com/questions/63380464
https://stackoverflow.com/questions/53017480
https://stackoverflow.com/questions/32809169


In [30]:
test_embeddings = encode_texts_to_embeddings(sentences=["Install Kubernetes on baremetal"])

In [31]:
NUM_NEIGHBOURS = 10

response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=test_embeddings,
    num_neighbors=NUM_NEIGHBOURS,
)

response

[[MatchNeighbor(id='55848559', distance=0.7050083875656128, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[]),
  MatchNeighbor(id='51188718', distance=0.6932722330093384, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[]),
  MatchNeighbor(id='56852615', distance=0.6908754110336304, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[]),
  MatchNeighbor(id='36819097', distance=0.6847041249275208, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[]),
  MatchNeighbor(id='59975211', distance=0.6828333139419556, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[]),
  MatchNeighbor(id='58023153', distance=0.6806507110595703, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[]),
  MatchNeighbor(id='54324599', distance=0.6755163073539734, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[]),
  MatchNeighbor(id='57973152', distance=0.670676589012146, fea

In [32]:
for match_index, neighbor in enumerate(response[0]):
    print(f"https://stackoverflow.com/questions/{neighbor.id}")

https://stackoverflow.com/questions/55848559
https://stackoverflow.com/questions/51188718
https://stackoverflow.com/questions/56852615
https://stackoverflow.com/questions/36819097
https://stackoverflow.com/questions/59975211
https://stackoverflow.com/questions/58023153
https://stackoverflow.com/questions/54324599
https://stackoverflow.com/questions/57973152
https://stackoverflow.com/questions/47866722
https://stackoverflow.com/questions/54788528
